In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, disk
from scipy.stats import mode
import matplotlib.pyplot as plt

In [ ]:
# To Use IHC 232 (ER POSITIVE)

# FOUR_CLASS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Combined_Loss/UNet 4 Class/232.npy"
# NEGATIVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Negative/Train/232.npy"


# To Use IHC 242 (ER NEGATIVE)
FOUR_CLASS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Combined_Loss/UNet 4 Class/242.npy"
NEGATIVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Negative/Test/242.npy"

In [ ]:
def getCountUtil(image):

  # Opening operation
  selem = disk(15)
  opened = opening(image, selem)

  # Finding the regions/nuclei
  nuclei = measure.label(opened)
  props = regionprops(nuclei)

  # Removing small pixel patches
  if(len(props)==0):
    return 0
  new_props = list.copy(props)
  for i in range(len(props)):
    if props[i].area <= 200:
      new_props.pop(i)
  props = new_props

  # Caclulating average nuclei size
  if(len(props)==0):
    return 0
  avg_size = 0
  for i in range(len(props)):
    avg_size += props[i].area 
  avg_size /= len(props)

  # Splitting Overlapping Nuclei and getting count
  count = len(props)
  for i in range(len(props)):
    if(props[i].area > avg_size):
      count += math.floor(props[i].area/avg_size - 1)  
  
  return count

In [ ]:
def getCount(image,intensity):
  new_image = np.copy(image)
  new_image[new_image!=intensity] = 0
  count = getCountUtil(new_image)
  return count

In [ ]:
def getIntensityScore(strongCount,intermediateCount,weakCount,positiveCount,negativeCount):
  IS = 0
  weak_percent = weakCount/(positiveCount + negativeCount)
  intermediate_percent = intermediateCount/(positiveCount + negativeCount)
  strong_percent = strongCount/(positiveCount + negativeCount)

  IS = weak_percent + intermediate_percent * 2 + strong_percent * 3
  # maxCount = max(strongCount,1.5 * intermediateCount,2 * weakCount)
  # if(maxCount == 0):
  #   IS = 0              # No Staining
  # elif(maxCount == 2* weakCount):
  #   IS = 1              # Weak Positive Staining
  # elif(maxCount == 1.5 *intermediateCount):
  #   IS = 2              # Moderate Positive Staining
  # else:
  #   IS = 3              # Strong Positive Staining
  return math.floor(IS)

In [ ]:
def getProliferationScore(positiveCount,negativeCount):
  PS = 0
  stainedCellPercentage = positiveCount / (positiveCount + negativeCount) * 100.0
  if(stainedCellPercentage == 0):
    PS = 0            # No cells are ER positive
  elif(stainedCellPercentage < 1):
    PS = 1            # <1% cells are ER positive
  elif(stainedCellPercentage <= 10):
    PS = 2            # 1-10% cells are ER positive
  elif(stainedCellPercentage <= 33):
    PS = 3            # 11-33% cells are ER positive
  elif(stainedCellPercentage <= 66):
    PS = 4            # 34-66% cells are ER positive
  else:
    PS = 5            # 67-100% cells are ER positive
  return PS

In [ ]:
def getAllredScore(IS,PS):
  return IS + PS      # Allred's score = Intensity Score + Proliferation Score

In [ ]:
def getResult(score):
  result = 0
  if(score<=2):
    result = 0      # 0-2: ER negative
  else:
    result = 1      # 3-8: ER positive
  return result

In [ ]:
def ScoreCalculation(mask,negativeMask):
  negativeCount = getCount(negativeMask,255)
  strongCount = getCount(mask,255)
  intermediateCount = getCount(mask,170)
  weakCount = getCount(mask,85)

  print("Negative : " , negativeCount)
  print("Strong : " , strongCount)
  print("Intermediate : " , intermediateCount)
  print("Weak : " , weakCount)

  positiveCount = strongCount + intermediateCount + weakCount

  IS = getIntensityScore(strongCount,intermediateCount,weakCount,positiveCount,negativeCount)
  PS = getProliferationScore(positiveCount,negativeCount)

  AS = getAllredScore(IS,PS)
  print("Score : " , AS , "\n\n\n")

  result = getResult(AS)
  return result

In [ ]:
# Patient-wise result
def patientWiseResult(fourClassNumpyPath,negativeNumpyPath):
  mask = np.load(fourClassNumpyPath)
  negativeMask = np.load(negativeNumpyPath)
  result = np.zeros(len(mask), dtype=int)
  
  for i in range(len(mask)):
    result[i] = ScoreCalculation(mask[i],negativeMask[i])
  return mode(result)

In [ ]:
val = patientWiseResult(FOUR_CLASS_PATH, NEGATIVE_PATH)
if(val):
  print("PROGNOSIS : ER POSITIVE")
else:
  print("PROGNOSIS : ER NEGATIVE")

Negative :  135
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  159
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  155
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  156
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  115
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  140
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  140
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  145
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  63
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



Negative :  70
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 



PROGNOSIS : ER POSITIVE
